<a href="https://colab.research.google.com/github/Shahnawaz2506/Recommendation/blob/main/Reco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel,BertConfig
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Load IMDb dataset and preprocess the text
# Assume you have loaded the data into X_train, X_test, y_train, y_test
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from google.colab import drive

# This will prompt you to click on a link, sign in to your Google account, and get an authorization code.
drive.mount('/content/drive')
drive_loc='/content/drive/My Drive/Colab Notebooks/Data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
# from transformers import BertTokenizer, TFBertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import csv

# Load your movie synopsis dataset
# Assuming the dataset is in a CSV file with a column 'synopsis'
dataset_path = drive_loc+'wiki_movie_plots_deduped.csv'  # Replace with your file path
movie_data = pd.read_csv(dataset_path,encoding='utf-8', error_bad_lines=False,quoting=csv.QUOTE_ALL)

<ipython-input-2-b1ef12bd1e9b>:10: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  movie_data = pd.read_csv(dataset_path,encoding='utf-8', error_bad_lines=False,quoting=csv.QUOTE_ALL)


In [ ]:

s=movie_data.Plot.apply(lambda x : len(x.split(" ")))//20
# s['index']=s['index']//10
s=s.value_counts().reset_index()
s['cum']=s.Plot.cumsum()
s

In [8]:
config = BertConfig.from_pretrained('bert-base-uncased')

# Set the dtype to float16 in the configuration
config.dtype = 'float16'
# Extract synopses
embeddings_final=np.array([])
for i in range(0,400,100):
    synopses = movie_data.loc[i:i+99,'Plot'].tolist()

    # Load BERT tokenizer and model
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = TFBertModel.from_pretrained('bert-base-uncased',config=config)

    # Tokenize and generate BERT embeddings for synopses
    max_length = 512  # Define your desired sequence length
    encoded_input = tokenizer(synopses, padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')

    # Generate BERT embeddings
    bert_outputs = bert_model(encoded_input)
    embeddings_int = (bert_outputs.pooler_output.numpy()*100).astype('int8')  # Assuming you're using the pooler_output
    if embeddings_final.size == 0:
        embeddings_final = embeddings_int
    else:embeddings_final=np.concatenate([embeddings_final,embeddings_int],axis=0)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [41]:
encoded_input2 = tokenizer(synopses, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt') #for inspection
input_ids = pd.DataFrame(encoded_input2['input_ids'].numpy())
input_ids['text']=synopses
# attention_mask = encoded_input2['attention_mask']
input_ids.to_csv('inspect_tokens.csv')
input_ids.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,101,101,101,101,101,101,101,101,101,101,101,101,101,101,101,101,101,101,101,101
1,1037,1996,1996,9879,1996,5650,1996,1996,1996,5019,1996,1996,3493,15033,2077,1996,2006,1037,1037,1037
2,15812,4231,2143,2074,5700,4076,2143,2143,3098,2024,4678,2143,2352,1005,5825,5436,1037,26599,2402,2317
3,2003,1010,1010,6079,2124,1037,7480,2003,3496,3107,16313,2838,2099,1055,2041,2003,3376,16222,3232,2611
4,2551,4993,2074,3823,6789,2312,2007,2055,3065,2478,10882,1037,14559,2684,2000,2008,2621,14122,7288,1006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,0,0,0,0,0,0,0,0,2174,0,0,0,0,0,0,0,0,0,0,0
509,0,0,0,0,0,0,0,0,1010,0,0,0,0,0,0,0,0,0,0,0
510,0,0,0,0,0,0,0,0,2031,0,0,0,0,0,0,0,0,0,0,0
511,0,0,0,0,0,0,0,0,102,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
for i,synopsis in enumerate(synopses): #fOR DEBUGGING
    print(synopsis[:10])
    if not synopsis.strip():  # Check if the string is empty or contains only whitespace
        print(i)


In [42]:

# Calculate pairwise cosine similarity between movie synopses
similarity_matrix = cosine_similarity(embeddings)

# Function to find similar movies based on indices
def find_similar_movies(movie_index, top_n=5):
    similarity_scores = list(enumerate(similarity_matrix[movie_index]))
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    top_similar = similarity_scores[1:top_n+1]  # Exclude the movie itself (most similar will be itself)
    similar_movies_indices = [movie[0] for movie in top_similar]
    return similar_movies_indices


                               Title  \
13                      Daniel Boone   
3   Terrible Teddy, the Grizzly King   
17                   The Black Viper   
6            The Great Train Robbery   
16          The Adventures of Dollie   

                                                 Plot  
13  Boone's daughter befriends an Indian maiden as...  
3   Lasting just 61 seconds and consisting of two ...  
17  A thug accosts a girl as she leaves her workpl...  
6   The film opens with two bandits breaking into ...  
16  On a beautiful summer day a father and mother ...  


In [ ]:

# Example: Find similar movies for a specific movie index (change index as needed)
movie_index = 0  # Change this index to find similar movies for a different movie
similar_movies_indices = find_similar_movies(movie_index)

# Get details of similar movies from the dataset
similar_movies = movie_data.iloc[similar_movies_indices]
print(similar_movies[['Title', 'Plot']])  # Change column names based on your dataset structure


In [29]:
# import numpy as np
from scipy.sparse import csr_matrix ,hstack
sparse_matrix_csr=csr_matrix((400,0))
# z=embeddings[:20,:]
for i in range(0,420,20):
# i=0
    cosine1=np.dot(embeddings_final*.01,embeddings_final[i:i+20,:].T*.01)
    # cosine1=np.dot(embed*.01,embed[:20,:].T*.01)
    top_indices = np.argsort(-cosine1, axis=0)[:5, :]

    # Create a mask matrix with zeros
    mask = np.zeros_like(cosine1, dtype=bool)

    # Set the top 5 indices in each column to True in the mask
    mask[top_indices, np.arange(cosine1.shape[1])] = True

    # Apply the mask to your original matrix
    masked_matrix = np.where(mask, cosine1, 0)
    sparse_matrix_csr1 = csr_matrix(masked_matrix)
    sparse_matrix_csr=hstack([sparse_matrix_csr,sparse_matrix_csr1])

pd.DataFrame(sparse_matrix_csr.toarray()).to_csv('emb1.csv')
masked_matrix

array([], shape=(400, 0), dtype=float64)

In [13]:
import sys
print(sys.getsizeof(masked_matrix))
print(sys.getsizeof(sparse_matrix_csr))

8128
48


In [51]:
sys.getsizeof(embeddings)

153728

In [40]:
embed=(embeddings*100).astype(np.int8)

In [19]:
import sys

# Get all variables in the global scope
all_variables = globals().items()

# Calculate the size of each variable and store it in a dictionary
variables_sizes = {name: sys.getsizeof(var) for name, var in all_variables}

# Sort the variables by size in descending order
sorted_variables = sorted(variables_sizes.items(), key=lambda x: x[1], reverse=True)

# Display the variables and their sizes
for name, size in sorted_variables:
    print(f"{name}: {size} bytes")


movie_data: 101849373 bytes
embeddings_final: 307328 bytes
_9: 307328 bytes
___: 76928 bytes
embeddings_int: 76928 bytes
_11: 76928 bytes
_12: 76928 bytes
_13: 76928 bytes
_14: 76928 bytes
_16: 76928 bytes
__: 64128 bytes
cosine1: 64128 bytes
masked_matrix: 64128 bytes
_17: 64128 bytes
mask: 8128 bytes
variables_sizes: 2272 bytes
BertTokenizer: 2016 bytes
_i8: 1091 bytes
_i7: 1089 bytes
Model: 1072 bytes
Adam: 1072 bytes
SparseCategoricalCrossentropy: 1072 bytes
_i5: 995 bytes
_i4: 993 bytes
TFBertModel: 904 bytes
BertConfig: 904 bytes
Dropout: 904 bytes
Dense: 904 bytes
csr_matrix: 904 bytes
synopses: 856 bytes
_15: 856 bytes
_i1: 806 bytes
_ii: 693 bytes
_i17: 693 bytes
sorted_variables: 600 bytes
_i6: 525 bytes
_i10: 525 bytes
_i19: 525 bytes
_i2: 496 bytes
bert_outputs: 424 bytes
_oh: 360 bytes
Out: 360 bytes
_ih: 248 bytes
In: 248 bytes
Input: 144 bytes
train_test_split: 144 bytes
cosine_similarity: 144 bytes
top_indices: 128 bytes
dataset_path: 122 bytes
__doc__: 113 bytes
drive_